In [2]:
import pandas as pd
import catboost as cb

In [3]:
# Чтение feed_data из csv файла
feed_data = pd.read_csv(r'C:\Users\Alex\Desktop\data_for_training.csv')
print(f"Feed data shape: {feed_data.shape}")

Feed data shape: (15378483, 51)


In [4]:
def sample_10_percent(group):
    frac = 0.5
    return group.sample(frac=frac)

feed_data = feed_data.groupby('user_id', group_keys=False).apply(sample_10_percent)

In [5]:
feed_data.columns

Index(['user_id', 'post_id_x', 'action', 'target', 'gender', 'age', 'country',
       'city', 'exp_group', 'os', 'source', 'topic', 'word_count',
       'sentence_count', 'avg_word_length', 'punctuation_count', 'month',
       'day_of_week', 'hour_of_day', 'time_since_last_action', 'day_of_month',
       'year', 'user_views', 'user_likes', 'post_views', 'post_likes',
       '0_exp_group_likes', '0_exp_group_views', '1_exp_group_likes',
       '1_exp_group_views', '2_exp_group_likes', '2_exp_group_views',
       '3_exp_group_likes', '3_exp_group_views', '4_exp_group_likes',
       '4_exp_group_views', '5_exp_group_likes', '5_exp_group_views',
       '6_exp_group_likes', '6_exp_group_views', 'component_1', 'component_2',
       'component_3', 'component_4', 'component_5', 'component_6',
       'component_7', 'component_8', 'component_9', 'component_10',
       'post_id_y'],
      dtype='object')

In [6]:
# rename post_id_x to post_id 
feed_data = feed_data.rename(columns={'post_id_x': 'post_id'})

# drop post_id_y
feed_data = feed_data.drop('post_id_y', axis=1)

# drop action
feed_data = feed_data.drop('action', axis=1)

# Print the first few rows of the DataFrame
print(feed_data.head())

# Print the summary statistics of the DataFrame
print(feed_data.describe())

# Print the information of the DataFrame
print(feed_data.info())

# Print the number of unique users
print("Number of unique users:", feed_data['user_id'].nunique())

# Print the number of unique posts
print("Number of unique posts:", feed_data['post_id'].nunique())

# Print the distribution of the number of posts per user
print("Number of posts per user:\n", feed_data['user_id'].value_counts())


In [7]:
cat_features  = ['gender', 'age', 'country', 'city', 'exp_group', 'os', 'source', 'topic']

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# Get a list of unique users
unique_users = feed_data['user_id'].unique()

# Split unique users into training and testing sets
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)

X_train = feed_data[feed_data['user_id'].isin(train_users)]
X_train = X_train.drop('target', axis=1)

X_test = feed_data[feed_data['user_id'].isin(test_users)]
X_test = X_test.drop('target', axis=1)

y_train = feed_data[feed_data['user_id'].isin(train_users)]['target']
y_test = feed_data[feed_data['user_id'].isin(test_users)]['target']


# Create a group ID based on the 'user_id' column

group_id_dict = {user_id: idx for idx, user_id in enumerate(unique_users)}
X_train['group_id'] = X_train['user_id'].map(group_id_dict) # map() function returns a map object(which is an iterator) of the results after applying the given function to each item of a given iterable (list, tuple etc.)
X_test['group_id'] = X_test['user_id'].map(group_id_dict)

# Sort the train and test sets by 'group_id'
X_train = X_train.sort_values(by='group_id')
y_train = y_train.loc[X_train.index] # loc() function is used to access a group of rows and columns by label(s) or a boolean array.

X_test = X_test.sort_values(by='group_id')
y_test = y_test.loc[X_test.index]

In [8]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# Get a list of unique users
unique_users = feed_data['user_id'].unique()

# Split unique users into training and testing sets
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)

X_train = feed_data[feed_data['user_id'].isin(train_users)]
X_train = X_train.drop('target', axis=1)

X_test = feed_data[feed_data['user_id'].isin(test_users)]
X_test = X_test.drop('target', axis=1)

y_train = feed_data[feed_data['user_id'].isin(train_users)]['target']
y_test = feed_data[feed_data['user_id'].isin(test_users)]['target']

# Create a group ID based on the 'user_id' column in the training set
group_id_dict_train = {user_id: idx for idx, user_id in enumerate(train_users)}
X_train['group_id'] = X_train['user_id'].map(group_id_dict_train)

# Create a group ID based on the 'user_id' column in the test set
group_id_dict_test = {user_id: idx for idx, user_id in enumerate(test_users)}
X_test['group_id'] = X_test['user_id'].map(group_id_dict_test)

# Sort the train and test sets by 'group_id'
X_train = X_train.sort_values(by='group_id')
y_train = y_train.loc[X_train.index]

X_test = X_test.sort_values(by='group_id')
y_test = y_test.loc[X_test.index]

# Create train and test Pool objects with the 'group_id' column
train_pool = Pool(X_train.drop(columns=['user_id']), y_train, group_id=X_train['group_id'])
test_pool = Pool(X_test.drop(columns=['user_id']), y_test, group_id=X_test['group_id'])


In [12]:
# Specify the training parameters 
params = {'loss_function': 'Logloss', # suitable for binary classification
          'custom_metric': ['AUC', 'NDCG'], # evaluation metrics
          'thread_count': 16,
          'verbose': 100,
          'random_seed': 42,
          'iterations': 1000,
          'learning_rate': 0.01,
          }

# Initialize CatBoostClassifier
model = CatBoostClassifier(**params)

# Fit model
model.fit(train_pool, eval_set=test_pool)

0:	learn: 0.6841068	test: 0.6842059	best: 0.6842059 (0)	total: 282ms	remaining: 4m 42s
100:	learn: 0.3607832	test: 0.3608844	best: 0.3608844 (100)	total: 27s	remaining: 4m
200:	learn: 0.3270229	test: 0.3271149	best: 0.3271149 (200)	total: 53.4s	remaining: 3m 32s
300:	learn: 0.3213309	test: 0.3214325	best: 0.3214325 (300)	total: 1m 19s	remaining: 3m 5s
400:	learn: 0.3193298	test: 0.3194465	best: 0.3194465 (400)	total: 1m 46s	remaining: 2m 38s
500:	learn: 0.3184363	test: 0.3185650	best: 0.3185650 (500)	total: 2m 12s	remaining: 2m 12s
600:	learn: 0.3178537	test: 0.3179899	best: 0.3179899 (600)	total: 2m 38s	remaining: 1m 45s
700:	learn: 0.3174319	test: 0.3175736	best: 0.3175736 (700)	total: 3m 5s	remaining: 1m 19s
800:	learn: 0.3171070	test: 0.3172561	best: 0.3172561 (800)	total: 3m 31s	remaining: 52.5s
900:	learn: 0.3168777	test: 0.3170353	best: 0.3170353 (900)	total: 3m 56s	remaining: 26s
999:	learn: 0.3166943	test: 0.3168609	best: 0.3168609 (999)	total: 4m 22s	remaining: 0us

bestTest 

In [13]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.drop(columns=['user_id']).columns

for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print(f'{name}: {score}')

age: 30.905743873401633
user_views: 19.139807644100106
post_views: 16.488681822328388
post_likes: 9.628621136365949
month: 5.231191510676601
user_likes: 4.787093854020484
time_since_last_action: 3.00661916953293
hour_of_day: 2.144813484544992
city: 1.7847359346798373
country: 1.6490598548654003
4_exp_group_likes: 0.5872170980684919
topic: 0.5302533427564742
1_exp_group_likes: 0.45781957325047157
gender: 0.40401728981105817
post_id: 0.38367674530640633
3_exp_group_views: 0.372007957144139
5_exp_group_likes: 0.3504371176586997
2_exp_group_likes: 0.33646112256309424
3_exp_group_likes: 0.2969667042724532
component_2: 0.27383448893953305
4_exp_group_views: 0.26688254309501613
0_exp_group_likes: 0.2643346503607561
6_exp_group_likes: 0.15641322658753923
1_exp_group_views: 0.10655812366124458
2_exp_group_views: 0.08864624762113689
exp_group: 0.0667488063263353
0_exp_group_views: 0.05753102742903716
component_4: 0.03585172410745325
5_exp_group_views: 0.03373393860632913
6_exp_group_views: 0.031

In [9]:
import matplotlib.pyplot as plt

feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.drop(columns=['user_id']).columns

sorted_scores, sorted_names = zip(*sorted(zip(feature_importances, feature_names), reverse=True))

plt.figure(figsize=(10, 6))
plt.bar(sorted_names, sorted_scores)
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance Score')
plt.title('Feature Importances')
plt.tight_layout()
plt.show()


NameError: name 'model' is not defined

In [10]:
# Train the CatBoost model using PrecisionAt:top=5 evaluation metric
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='PrecisionAt:top=5',
                           eval_metric='PrecisionAt:top=5',
                           random_seed=42,
                           verbose=100)

model.fit(train_pool, eval_set=test_pool)

0:	learn: 0.0670078	test: 0.0670384	best: 0.0670384 (0)	total: 253ms	remaining: 4m 12s
100:	learn: 0.1732239	test: 0.1734873	best: 0.1735363 (97)	total: 23s	remaining: 3m 25s
200:	learn: 0.1739377	test: 0.1732913	best: 0.1738979 (147)	total: 45.3s	remaining: 2m 59s


: 

: 

In [9]:
y_pred_proba = model.predict_proba(test_pool)[:, 1]
# Add the prediction probabilities to the test dataset
X_test['pred_proba'] = y_pred_proba

# Group by 'user_id' and find the top 5 predicted 'post_id' for each user
top_5_posts = X_test.groupby('user_id').apply(lambda x: x.nlargest(5, 'pred_proba')['post_id'])

In [10]:
top_5_posts

user_id          
200      75          5181
         37          6404
         57          3890
         38          1936
         23          5989
                     ... 
168551   15378387    2474
         15378377     361
         15378390    4217
         15378375    1669
         15378388    5691
Name: post_id, Length: 163205, dtype: int64

In [11]:
def hitrate_at_k(top_k_posts, k=5):
    hits = 0
    user_data = top_k_posts.groupby(level=0).apply(list).to_dict()

    for user_id, top_k_posts in user_data.items():
        true_post_ids = X_test[(X_test['user_id'] == user_id) & (y_test == 1)]['post_id']
        hits += len(set(top_k_posts) & set(true_post_ids))

    return hits / len(y_test[y_test == 1])

hitrate_at_5 = hitrate_at_k(top_5_posts, k=5)
print("HitRate@5:", hitrate_at_5)


HitRate@5: 0.09368657129728529


In [ ]:
# Save the model to a file
model.save_model("catboost_model.cbm")